In [1]:
import os

In [2]:
!pwd

/home/gom/Workspace/jobsum/research


In [3]:
os.chdir('../')
!pwd

/home/gom/Workspace/jobsum


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

Imported O.K!


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
import torch
from datasets import load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


/home/gom/miniconda3/envs/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        def convert_examples_to_features(example_batch):
            input_encodings = tokenizer(example_batch['dialogue'],
                                        max_length=128, #1024
                                        truncation = True )

            with tokenizer.as_target_tokenizer():
                target_encodings = tokenizer(
                    example_batch['summary'],
                    max_length=64, # 128
                    truncation=True
                )

            return{
                'input_ids': input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum_pt.map(convert_examples_to_features, batched = True)

        # init arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16
        )

        # init trainer
        trainer = Trainer(model=model_pegasus,
                          args=trainer_args,
                          tokenizer=tokenizer,
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt['train'],
                          eval_dataset=dataset_samsum_pt['validation'])
        
        # train model
        trainer.train()

        # save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,\
                                                   'pegasus-samsum-model'))
        
        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))


In [9]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer_config = ModelTrainer(config=model_trainer_config)
model_trainer_config.train()

[2023-11-19 20:05:00,832: INFO:      common: yaml file: config/config.yaml loaded successfully]
[2023-11-19 20:05:00,834: INFO:      common: yaml file: params.yaml loaded successfully]
[2023-11-19 20:05:00,834: INFO:      common: created directory at: artifacts]
[2023-11-19 20:05:00,834: INFO:      common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map:   0%|          | 0/819 [00:00<?, ? examples/s]/home/gom/miniconda3/envs/env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
  1%|          | 10/920 [00:16<25:12,  1.66s/it]

{'loss': 3.2174, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/920 [00:33<25:07,  1.68s/it]

{'loss': 3.4153, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


...

{'loss': 1.7653, 'learning_rate': 2.3809523809523808e-06, 'epoch': 0.98}


 99%|█████████▉| 910/920 [26:00<00:16,  1.70s/it]

{'loss': 1.7961, 'learning_rate': 1.1904761904761904e-06, 'epoch': 0.99}


100%|██████████| 920/920 [26:17<00:00,  1.71s/it]


{'loss': 1.7939, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 1577.1448, 'train_samples_per_second': 9.341, 'train_steps_per_second': 0.583, 'train_loss': 1.996659981686136, 'epoch': 1.0}
